# K-Nearest Neighbor Lab





In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import arff
from IPython.core.display import display

import math
import copy

from sklearn.preprocessing import LabelEncoder
import pdb

## 1. (40%) Correctly implement the k-nearest neighbor (KNN) algorithm and the KNN regression algorithm

### Code requirements
- Use Euclidean distance to decide closest neighbors. 
- Include optional distance weighting for both algorithms


In [2]:
class KNNClassifier(BaseEstimator,ClassifierMixin):
    def __init__(self, columntype=[], weight_type='inverse_distance', knn=3, normalize=False): ## add parameters here
        """
        Args:
            columntype for each column tells you if continues[real] or if nominal[categoritcal].
            weight_type: inverse_distance voting or if non distance weighting. Options = ["no_weight","inverse_distance"]
        """
        self.columntype = columntype #Note This won't be needed until part 5
        self.weight_type = weight_type
        self.knn = knn
        self.normalize = normalize

    def fit(self, X, y, labels=None):
        """ Fit the data; run the algorithm (for this lab really just saves the data :D)
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 2D numpy array with the training targets
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        self.X, self.y = X, y
        return self
    
    def predict(self, X):
        """ Predict all classes for a dataset X
        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """

        predictions = np.zeros(X.shape[0])

        for i in range(X.shape[0]):
            instance = X[i, :]
            distances = np.zeros(self.X.shape[0])

            for j in range(self.X.shape[0]):
                other = self.X[j, :]
                distance = self._distance(instance, other)
                distances[j] = distance
            predicted_class = self._predicted_class(copy.deepcopy(self.y), copy.deepcopy(distances))
            predictions[i] = predicted_class

        return predictions

    def _distance(self, instance, other):
        distances = np.zeros(instance.shape)
        for i in range(len(instance)):
            if self.columntype[i] == 'nominal':
                if other[i] == instance[i]:
                    distances[i] = 0
                else:
                    distances[i] = 1
            else:
                distances[i] = (other[i] - instance[i])**2
            
        sm = np.sum(distances)
        return sm**(1/2)


    def _predicted_class(self, classes, distances):

        data = np.append(classes.reshape(-1, 1), distances.reshape(-1, 1), axis=1)
        data = data[data[:, 1].argsort()]
        data = data[:self.knn, :]

        distances = data[:, 1]
        classes = data[:, 0]

        if self.weight_type == 'inverse_distance':
            data[:, 1] = np.where(data[:, 1] != 0, 1/(data[:, 1]**2), np.inf)
        else:
            data[:, 1] = 1

        df = pd.DataFrame(data=data, columns=['class', 'vote'])
        df = df.groupby(by='class').agg({'vote': 'sum'}) \
            .sort_values(by='vote', ascending=False) \
            .reset_index()

        return int(df.iloc[0, 0])


    #Returns the Mean score given input data and labels
    def score(self, X, y):
        """ Return accuracy of model on a given dataset. Must implement own score function.
        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 2D numpy array with targets
        Returns:
            score : float
                Mean accuracy of self.predict(X) wrt. y.
        """
        predictions = self.predict(X)
        return (predictions == y.reshape(-1)).mean()


In [3]:
def load_data(filename):
    data = arff.loadarff(filename)
    df = pd.DataFrame(data[0])

    for i in range(len(df.dtypes)):
        if df.dtypes.astype(str).iloc[i] == 'object':
            column = df.columns[i]
            df[column] = df[column] \
                            .astype(str).str \
                            .split("\'", expand=True) \
                            .iloc[:,1]
    return df

def encode_X(X1, X2):
    columns = X1.columns

    for column in columns:
        if X1[column].dtype == 'object':
            le = LabelEncoder()
            le.fit(X1[column])
            X1[column] = le.transform(X1[column])
            X2[column] = le.transform(X2[column])
   
    return X1, X2

def encode_y(y1, y2):
    name = y1.name
    le = LabelEncoder()
    le.fit(y1)
    y1 = le.transform(y1)
    y2 = le.transform(y2)
    return pd.DataFrame(y1, columns=[name]), pd.DataFrame(y2, columns=[name])

def get_column_types(df):
    return df.dtypes.astype(str) \
            .map({'float64': 'continuous', 'object': 'nominal'}) \
            .tolist()

## 1.1 Debug and Evaluation

Debug and Evaluate your model using the parameters below:

- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance

---

### 1.1.1 Debug

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/glass_test.arff)
- Use distance weighting
- KNN = 3 (three nearest neighbors)
- Don’t normalize the data
- Use Euclidean Distance

Expected Results:
- Not using inverse weighted distancing = roughly [68.29%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_no_inv_prediction.csv)

- Using inverse weighted distancing = roughly [74.39%]
- Link to [debug solution](https://github.com/cs472ta/CS472/blob/master/debug_solutions/glass_inv_prediction.csv)


In [4]:
# Load glass data
glass_trainDF = load_data('datasets/glass_train.arff')
glass_testDF = load_data('datasets/glass_test.arff')

# Get column types
column_types = get_column_types(glass_trainDF)[:-1]
print(column_types)

glass_X_trainDF, glass_y_trainDF = glass_trainDF.iloc[:, :-1], glass_trainDF.iloc[:, -1]
glass_X_testDF, glass_y_testDF = glass_testDF.iloc[:, :-1], glass_testDF.iloc[:, -1]

glass_X_train, glass_X_test = encode_X(glass_X_trainDF, glass_X_testDF)
glass_y_train, glass_y_test = encode_y(glass_y_trainDF, glass_y_testDF)

display(glass_X_train.head())
display(glass_y_train.head())
display(glass_X_test.head())
display(glass_y_test.head())

X_train = glass_X_train.to_numpy()
X_test = glass_X_test.to_numpy()
y_train = glass_y_train.to_numpy()
y_test = glass_y_test.to_numpy()

# Train on training set
glass_knn_d_weighting = KNNClassifier(columntype=column_types)
glass_knn_nd_weighting = KNNClassifier(columntype=column_types, weight_type=None)
glass_knn_d_weighting.fit(X_train, y_train)
glass_knn_nd_weighting.fit(X_train, y_train)

# Predict on test set
glass_score_d_weighting = glass_knn_d_weighting.score(X_test, y_test)
glass_score_nd_weighting = glass_knn_nd_weighting.score(X_test, y_test)
print("Score with inverted distance weighting: %.4f" % glass_score_d_weighting)
print("Score with no inverted distance weighting: %.4f" % glass_score_nd_weighting)

['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous']


,RI,Na,Mg,Al,Si,'K',Ca,Ba,Fe
0,1.51793,12.79,3.50,1.12,73.03,0.64,8.77,0.0,0.00
1,1.51643,12.16,3.52,1.35,72.89,0.57,8.53,0.0,0.00
2,1.51793,13.21,3.48,1.41,72.64,0.59,8.43,0.0,0.00
3,1.51299,14.40,1.74,1.54,74.55,0.00,7.59,0.0,0.00
4,1.53393,12.30,0.00,1.00,70.16,0.12,16.19,0.0,0.24


,Type
0,0
1,5
2,0
3,4
4,1


,RI,Na,Mg,Al,Si,'K',Ca,Ba,Fe
0,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
2,1.51763,12.61,3.59,1.31,73.29,0.58,8.50,0.00,0.0
3,1.51596,13.02,3.56,1.54,73.11,0.72,7.90,0.00,0.0
4,1.51674,12.79,3.52,1.54,73.36,0.66,7.90,0.00,0.0


,Type
0,3
1,0
2,0
3,1
4,1


Score with inverted distance weighting: 0.7439
Score with no inverted distance weighting: 0.6829


### 1.1.2 Evaluate

We will evaluate your model based on its performance on the [diabetes](https://archive.ics.uci.edu/ml/datasets/Diabetes) problem.
- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/diabetes_test.arff)

In [5]:
# Load diabetes data
diabetes_trainDF = load_data('datasets/diabetes_train.arff')
diabetes_testDF = load_data('datasets/diabetes_test.arff')

display(diabetes_trainDF.head(2))
display(diabetes_testDF.head(2))

# Get column types
column_types = get_column_types(diabetes_trainDF)[:-1]
print(column_types)

diabetes_X_trainDF, diabetes_y_trainDF = diabetes_trainDF.iloc[:, :-1], diabetes_trainDF.iloc[:, -1]
diabetes_X_testDF, diabetes_y_testDF = diabetes_testDF.iloc[:, :-1], diabetes_testDF.iloc[:, -1]

diabetes_X_train, diabetes_X_test = encode_X(diabetes_X_trainDF, diabetes_X_testDF)
diabetes_y_train, diabetes_y_test = encode_y(diabetes_y_trainDF, diabetes_y_testDF)

# Train on training set
X_train = diabetes_X_train.to_numpy()
X_test = diabetes_X_test.to_numpy()
y_train = diabetes_y_train.to_numpy()
y_test = diabetes_y_test.to_numpy()

diabetes_knn = KNNClassifier(columntype=column_types)
diabetes_knn.fit(X_train, y_train)

# Predict on test set
score = diabetes_knn.score(X_test, y_test)
print("Evaluation dataset KNN accuracy: %.3f" % score)


,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative


,preg,plas,pres,skin,insu,mass,pedi,age,class
0,4.361603,151.490212,72.734913,35.139227,0.717017,36.578002,4.415094,53.908127,tested_positive
1,4.747226,88.035457,60.971079,32.756773,-5.969642,25.839988,7.309291,23.737320,tested_negative


['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous']
Evaluation dataset KNN accuracy: 0.891


## 2. (10%) Use the k-nearest neighbor algorithm (without distance weighting) for the [magic telescope](http://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope) problem

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/magic_telescope_test.arff) 

### 2.1
- Try it with k=3 and without normalization


In [6]:
# Load diabetes data
magic_telescope_trainDF = load_data('datasets/magic_telescope_train.arff')
magic_telescope_testDF = load_data('datasets/magic_telescope_test.arff')

display(magic_telescope_trainDF.head(2))
display(magic_telescope_testDF.head(2))

# Get column types
column_types = get_column_types(magic_telescope_trainDF)[:-1]
print(column_types)

magic_telescope_X_trainDF, magic_telescope_y_trainDF = magic_telescope_trainDF.iloc[:, :-1], magic_telescope_trainDF.iloc[:, -1]
magic_telescope_X_testDF, magic_telescope_y_testDF = magic_telescope_testDF.iloc[:, :-1], magic_telescope_testDF.iloc[:, -1]

magic_telescope_X_train, magic_telescope_X_test = encode_X(magic_telescope_X_trainDF, magic_telescope_X_testDF)
magic_telescope_y_train, magic_telescope_y_test = encode_y(magic_telescope_y_trainDF, magic_telescope_y_testDF)

# Train on training set
X_train = magic_telescope_X_train.to_numpy()
X_test = magic_telescope_X_test.to_numpy()
y_train = magic_telescope_y_train.to_numpy()
y_test = magic_telescope_y_test.to_numpy()

magic_telescope_knn = KNNClassifier(columntype=column_types)
magic_telescope_knn.fit(X_train, y_train)

# Predict on test set
score = magic_telescope_knn.score(X_test, y_test)
print("Magic telescope dataset KNN accuracy: %.3f" % score)


,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,22.7815,14.9526,2.4362,0.4982,0.2509,-14.2836,-9.3635,13.0939,3.0779,141.562,g
1,40.6756,15.5940,2.7447,0.2772,0.1449,19.6226,9.0297,7.4157,15.4260,193.234,g


,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,30.5540,21.1807,2.7509,0.2715,0.1500,2.1798,-20.8938,-15.2262,2.2982,197.390,g
1,59.2444,10.4634,2.8051,0.4019,0.1971,49.4360,25.1317,7.0386,71.0828,48.129,h


['continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous', 'continuous']


<ipython-input-2-237eb440ff45>:73: RuntimeWarning: divide by zero encountered in true_divide
  data[:, 1] = np.where(data[:, 1] != 0, 1/(data[:, 1]**2), np.inf)


Magic telescope dataset KNN accuracy: 0.809


### 2.2
- Try it with k=3 and with normalization (input features normalized between 0 and 1). Use the normalization formula (x-xmin)/(xmax-xmin)

In [7]:
# Train/Predict with normalization


*Discuss the accuracy results of using normalized data vs. unnormalized data*

### 2.3

- Using your normalized data, create one graph with classification accuracy on the test set over k values. 
    - Use odd values of k from 1 to 15.
- As a rough sanity check, typical knn accuracies for the magic telescope data set are 75-85%

In [8]:
# Train/Predict with normalization using k=1,3,...,15

# Graph classification accuracy over k


# For the rest of the experiments use only normalized data

## 3. (10%) Use the regression variation of your algorithm (without distance weighting) for the [housing price prediction](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) problem.

- Use this [training set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_train.arff) and this [test set](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/housing_test.arff).
- Use Mean Square Error (MSE) on the test set as your accuracy metric for this case.
    - Do not normalize regression output values
- Graph MSE on the test set with odd values of k from 1 to 15


In [9]:
# Load housing price prediction data

# Train/Predict using k=1,3,...,15

# Graph MSE over k


## 4. (15%) Repeat your experiments for magic telescope and housing using distance-weighted (inverse of distance squared) voting and discuss your results.


## 4.1 Magic Telescope Dataset

In [10]:
# Train/Predict magic telescope using distance-weighted voting

## 4.2 Housing Dataset

In [11]:
# Train/Predict housing using distance-weighted voting

*Discuss your results*

## 5. (10%) Use the k-nearest neighbor algorithm to solve the [credit-approval](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) (credit-a) problem.

- Use this [dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/credit_approval.arff)
    - Use a 70/30 split of the data for the training/test set
- Note that this set has both continuous and nominal attributes, together with don’t know values. 
- Implement and justify a distance metric which supports continuous, nominal, and don’t know attribute values
    - You need to handle don't knows with the distance metric, not by imputing a value.
    - More information on distance metrics can be found [here](https://www.jair.org/index.php/jair/article/view/10182/24168).
- Use your own choice for k.
- As a rough sanity check, typical knn accuracies for the credit data set are 70-80%.


In [12]:
# Load dataset and split into train/test sets
creditDF = load_data('datasets/credit_approval.arff')
credit_X_train, credit_X_test, credit_y_train, credit
display(credit_XDF.head(2))
display(credit_yDF)



# Train/Predict credit-approval


NameError: name 'credit_X_train' is not defined

*Explain and justify your distance metric*

## 6. (15%) Use the scikit's KNN Classifier on magic telescope and KNN Regressor on housing and compare your results.

- Try out different hyperparameters to see how well you can do. 


In [ ]:
# Train/Predict magic telescope using scikit's KNN

# Train/Predict housing using scikit's KNN


*Report your comparison*

## 7. (optional 5% extra credit): For the best value of k for any one of the datasets, implement a reduction algorithm that removes data points in some rational way such that performance does not drop too drastically on the test set given the reduced training set.

- Compare your performance on the test set for the reduced and non-reduced versions and give the number (and percentage) of training examples removed from the original training set. How well does your reduction algorithm work?
    - Note that performance for magic telescope is classification accuracy and for housing it is mean squared error.
    - Magic Telescope has about 12,000 instances and if you use a leave one out style of testing for your data set reduction, then your algorithm will run slow since that is n2 at each step.
    - If you wish, you may use a random subset of 2,000 of the magic telescope instances.
    - More information on reduction techniques can be found [here](http://axon.cs.byu.edu/~martinez/classes/478/slides/IBL.pdf).
